In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'prod'

In [ ]:
import src.config
from deltalake import DeltaTable
from pathlib import Path
import pandas as pd


In [ ]:
TABLE_PATH = src.config.BRONZE / "metadata_lookup_engine1"
metadata = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
metadata

In [ ]:
# TABLE_PATH = src.config.BRONZE / "metadata_lookup_engine1"
TABLE_PATH = src.config.SILVER / "account"


In [ ]:
account = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
account

In [ ]:
account_cleaned = account[
    (account['mjaccttypcd'].isin(['CML','MTG','MLN']))
].copy()

In [ ]:
account_cleaned

In [ ]:
customer_address_link = DeltaTable(src.config.SILVER / "customer_address_link").to_pandas()

In [ ]:
customer_address_link

In [ ]:
customer_address_link = customer_address_link[customer_address_link['addrusecd'] == 'PRI'].copy()


In [ ]:
customer_address_link = customer_address_link[[
    'customer_id',
    'addrnbr'
]].copy()

In [ ]:
account_cleaned = account_cleaned.merge(customer_address_link, on='customer_id', how='left')

In [ ]:
TABLE_PATH = src.config.SILVER / "address"
address = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
address = address.drop(columns='load_timestamp_utc').copy()

In [ ]:
import cdutils.input_cleansing

In [ ]:
account_cleaned_schema = {
    'addrnbr':'str'
}
account_cleaned = cdutils.input_cleansing.cast_columns(account_cleaned, account_cleaned_schema)

In [ ]:
customer_address_link_schema = {
    'addrnbr':'str'
}
customer_address_link = cdutils.input_cleansing.cast_columns(customer_address_link, customer_address_link_schema)

In [ ]:
account_cleaned = account_cleaned.merge(address, on='addrnbr', how='left')

In [ ]:
account_cleaned

In [ ]:
account_cleaned

In [ ]:
TABLE_PATH = src.config.SILVER / "property"
property = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
property

In [ ]:
property = property.drop(columns='load_timestamp_utc').copy()

In [ ]:
# Acct prop linking
TABLE_PATH = src.config.SILVER / "account_property_link"
acct_prop_link = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
acct_prop_link

In [ ]:
acct_prop_link = acct_prop_link.drop(columns='load_timestamp_utc').copy()

In [ ]:
account_cleaned.info()

In [ ]:
acct_prop_link.info()

In [ ]:
merged_df = account_cleaned.merge(acct_prop_link, how='inner', on='acctnbr').merge(property, how='left', on='propnbr')

In [ ]:
merged_df

In [ ]:
heloc_non_flood_to_exclude = ['MG52','MG55','MG48','MG71']
mask_to_exclude = (
    (merged_df['currmiaccttypcd'].isin(heloc_non_flood_to_exclude)) &
    (merged_df['floodzoneyn'] == 'N')
)

In [ ]:
# Filter out HELOCs that are not in flood zones
merged_df = merged_df[~mask_to_exclude].copy()

In [ ]:
merged_df

In [ ]:
TABLE_PATH = src.config.SILVER / "insurance"
insurance = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
insurance

In [ ]:
insurance['instypcd'].unique()

In [ ]:
# Exclude PMI
insurance = insurance[insurance['instypcd'] != "PMI"].copy()

In [ ]:
insurance.info(verbose=True)

In [ ]:
insurance = insurance[[
    'intrpolicynbr',
    'instypcd',
    'instypdesc',
    'insorgnbr',
    'escrowyn_link',
    'premamt',
    'effdate_policy',
    'expiredate',
    'inactivedate_policy',
    'coverageamt'
]].copy()

In [ ]:
insurance.info()

In [ ]:
insurance['insorgnbr'] = insurance['insorgnbr'].astype(str)

In [ ]:
TABLE_PATH = src.config.BRONZE / "wh_org"
wh_org = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
wh_org['orgnbr'] = wh_org['orgnbr'].astype(str)

In [ ]:
wh_org = wh_org[['orgnbr','orgname']].copy()

In [ ]:
wh_org = wh_org.rename(columns={'orgname':'Insurance Company Name'}).copy()

In [ ]:
insurance = insurance.merge(wh_org, how='left', left_on='insorgnbr', right_on='orgnbr')

In [ ]:
insurance

In [ ]:
insurance = insurance.drop(columns=['insorgnbr','orgnbr']).copy()

In [ ]:
TABLE_PATH = src.config.SILVER / "acct_prop_ins_link"
acct_prop_ins_link = DeltaTable(TABLE_PATH).to_pandas()

In [ ]:
acct_prop_ins_link

In [ ]:
acct_prop_ins_link.info()

In [ ]:
acct_prop_ins_link = acct_prop_ins_link[['propnbr','intrpolicynbr']].copy()
acct_prop_ins_link = acct_prop_ins_link.drop_duplicates().reset_index(drop=True)

In [ ]:
acct_prop_ins_link

In [ ]:
merged_df.info(verbose=True)

In [ ]:
merged_df = merged_df[[
    'acctnbr',
    'ownersortname',
    'notebal',
    'Net Balance',
    'noteopenamt',
    'orig_ttl_loan_amt',
    'Full_Street_Address',
    'cityname',
    'statecd',
    'zipcd',
    'propnbr',
    'proptypcd',
    'proptypdesc',
    'propaddr1',
    'propaddr2',
    'propaddr3',
    'propcity',
    'propstate',
    'propzip',
    'propdesc',
    'floodzoneyn',
    'floodzone',
    'aprsvalueamt',
    'product',
    'mjaccttypcd',
    'currmiaccttypcd',
    'creditlimitamt',
    'contractdate'
]].copy()

In [ ]:
merged_df = merged_df.rename(columns={
    'Full_Street_Address':'Borrower Primary Address',
    'cityname':'Borrower City',
    'statecd':'Borrower State',
    'zipcd': 'Borrower Zip'
}
).copy()

In [ ]:
new_merged_df = merged_df.merge(acct_prop_ins_link, how='left',on='propnbr').merge(insurance, how='left', on='intrpolicynbr')

In [ ]:
new_merged_df

In [ ]:
# Property type grouping configuration
PROPERTY_TYPE_GROUPS = {
    'Autobody/Gas Station': ['Autobody/Gas Station','Gas Station and Convenience St','Auto-Truck Repair','Car Wash'],
    'Retail': ['Retail - Big Box Store','Shopping Plaza','Strip Plaza','General Retail','Dealership'],
    'Hospitality': ['Hotel/Motel','Hospitality/Event Space','Assisted Living'],
    'Recreation': ['Outdoor Recreation','Indoor Recreational','Golf Course','Marina'],
    'Industrial': ['Manufacturing','Warehouse','Industrial','Seafood Processing Plant','Solar Farm'],
    'Land': ['Land - Unimproved','Land - Improved','Parking Lot'],
    'Mixed Use': ['Mixed Use (Retail/Office)','Mixed Use (Retail/Residential)','Mixed Use (Office/Residential)'],
    'Multi Family': ['Apartment Building','Multi Family'],
    'General Office': ['Office - Professional','Office- General'],
    'Medical Office': ['Office - Medical'],
    'Restaurant': ['Restaurant'],
    'Residential': ['1-4 Fam Res - Non Own Occ','1 Family Residential - Own Occ','2 Family Residential - Own Occ','Condominium'],
    'Storage': ['Self Storage'],
    'Educational': ['Educational Facilities','Day Care'],
    'Religious': ['Church'],
    'Vehicles': ['Vehicle - Business','Boat'],
    'Other': ['Commercial - Other','Real Estate - Business','Real Estate - Bus&Bus Assets','Real Estate - Personal & Bus','Real Estate - Pers&Bus Assets','All Business Assets','Bus Assets w/Accts Receivable','UCC - ABA','UCC- Equipment','Assignment of Leases/Rents','General Contractor','Outdoor Dealers','Marketable Securities','SBA Loan','Funeral Home','Savings - Partially Secured','Passbook/Savings Secured']
}



In [ ]:
# Create mapping from individual property types to groups
proptype_mapping = {code: group for group, codes in PROPERTY_TYPE_GROUPS.items() for code in codes}

# Add cleaned property type to property data
new_merged_df['Cleaned Prop Type'] = new_merged_df['proptypdesc'].map(proptype_mapping).fillna('Other')

In [ ]:
new_merged_df.info(verbose=True)

In [ ]:
new_merged_df

In [ ]:
acct_role_link = DeltaTable(src.config.SILVER / "acct_role_link").to_pandas()

In [ ]:
acct_role_link = acct_role_link[acct_role_link['acctrolecd'] == 'OWN']
acct_role_link = acct_role_link[[
    'acctnbr',
    'customer_id'
]].copy()
acct_role_link = acct_role_link.drop_duplicates(subset=['acctnbr'], keep='first')

In [ ]:
base_cust_dim = DeltaTable(src.config.SILVER / "base_customer_dim").to_pandas()

In [ ]:
base_cust_dim

In [ ]:
base_cust_dim = base_cust_dim[[
    'customer_id',
    'customer_name'
]].copy()

In [ ]:
base_cust_dim = base_cust_dim.rename(columns={
    'customer_name':'Primary Borrower 2'
}).copy()
acct_role_link = acct_role_link.merge(base_cust_dim, on='customer_id', how='left')

In [ ]:
acct_role_link

In [ ]:
acct_role_link = acct_role_link[[
    'acctnbr',
    'Primary Borrower 2'
]].copy()

In [ ]:
new_merged_df = new_merged_df.merge(acct_role_link, on='acctnbr', how='left')

In [ ]:
new_merged_df

In [ ]:
new_merged_df.info()

In [ ]:
new_merged_df = new_merged_df.rename(columns={
    'acctnbr':'Loan_Number',
    'ownersortname':'Borrower_Name',
    'Primary Borrower 2':'Borrower_Name_2',
    'Borrower Primary Address':'Borrower_Street_1',
    # Null row for Borrower Street 2
    'Borrower City':'Borrower_City',
    'Borrower State':'Borrower_State',
    'Borrower Zip':'Borrower_Zip',
    'notebal':'Principal_Balance',
    'creditlimitamt':'Line_Of_Credit_Amount',
    'contractdate':'Origination_Date',
    # Calculated Field for Collateral_Type = 'Real Estate'
    'propaddr1':'Property_Street_1',
    'propaddr2':'Property_Street_2',
    'propcity':'Property_City',
    'propstate':'Property_State',
    'propzip':'Property_Zip',
    'coverageamt':'Insurable_Value',
    # Replacement_Cost null field
    'floodzone':'Flood_Zone', # SWAP to actual flood zone description
    'instypcd':'Coverage_Type',
    'escrowyn_link':'Premium_Escrowed',
    'mjaccttypcd':'Loan_Type',
    'proptypdesc':'Building_Type',
    # ---
    'propdesc':'Collateral_Description'
}).copy()

new_merged_df['Collateral_Type'] =  'Real Estate'
new_merged_df['Borrower_Street_2'] = None
new_merged_df['Replacement_Cost'] = None

cleaned_df = new_merged_df[[
'Loan_Number',
'Borrower_Name',
'Borrower_Name_2',
'Borrower_Street_1',
'Borrower_Street_2',
'Borrower_City',
'Borrower_State',
'Borrower_Zip',
'Principal_Balance',
'Line_Of_Credit_Amount',
'Origination_Date',
'Collateral_Type',
'Property_Street_1',
'Property_Street_2',
'Property_City',
'Property_State',
'Property_Zip',
'Insurable_Value',
'Replacement_Cost',
'Flood_Zone',
'Coverage_Type',
'Premium_Escrowed',
'Loan_Type',
'Building_Type',
# Additional fields not on requested template
'Collateral_Description',
'currmiaccttypcd',
'product'
]].copy()


In [ ]:
cleaned_df

In [ ]:


# List of terms to exclude (case insensitive)
exclude_terms = [
    "All Business Assets",
    "UCC",
    "Assign",
    "Land",
    "Vehicle",
    "Equipment",
    "Bus Assets w/Acct Receivable",
    "Bus Assets",
    "Boat",
    "Passbook",
    "Certificate of Deposit",
    "Cash Life Insurance",
    "Parking Lot",
    "Marketable Securities",
    "Accounts Receivable",
    "Savings",
    "Classic Auto",
    "Inventory",
    "Mobile Home",
    "Lease Hold Mortgage",
    "Key Person life insurance",
    "security agreement",
    "stock",
    "Vessel"
]

# Function to check if any exclude term is in the collateral type (case insensitive)
def should_exclude(collateral_type):
    ct_lower = collateral_type.lower()
    return any(term.lower() in ct_lower for term in exclude_terms)

# Apply the filter
cleaned_df = cleaned_df[~cleaned_df['Building_Type'].apply(should_exclude)]


In [ ]:
OUTPUT_DIR = Path('./output')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
filename = 'lee_mason_extract.xlsx'
OUTPUT_PATH = OUTPUT_DIR / filename
cleaned_df.to_excel(OUTPUT_PATH, index=False) 